## Understanding and Predicting Property Maintenance Fines,
   This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 
   
   The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?
    
    The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.
    
  All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:
 *  
    * [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf),
    * [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv),
    * [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn),
    * [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3),
    * [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)
    
    We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.
    ,
    Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.
    
    <br>
    **File descriptions** (Use only this data for training your model!)
    
        train.csv - the training set (all tickets issued 2004-2011),
        test.csv - the test set (all tickets issued 2012-2016),
        addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates., 
    <br>
    **Data fields**,
    
    train.csv & test.csv,
    
        ticket_id - unique identifier for tickets,
        agency_name - Agency that issued the ticket,
        inspector_name - Name of inspector that issued the ticket,
        violator_name - Name of the person/organization that the ticket was issued to,
        violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred,
        mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator,
        ticket_issued_date - Date and time the ticket was issued,
        hearing_date - Date and time the violator's hearing was scheduled,
        violation_code, violation_description - Type of violation,
        disposition - Judgment and judgement type,
        fine_amount - Violation fine amount, excluding fees,
        admin_fee - $20 fee assigned to responsible judgments,
    state_fee - $10 fee assigned to responsible judgments,
        late_fee - 10% fee assigned to responsible judgments,
        discount_amount - discount applied, if any,
        clean_up_cost - DPW clean-up or graffiti removal cost,
        judgment_amount - Sum of all fines and fees,
        grafitti_status - Flag for graffiti violations,
        
    train.csv only,
   
        payment_amount - Amount paid, if any,
        payment_date - Date payment was made, if it was received,
        payment_status - Current payment status as of Feb 1 2017,
        balance_due - Fines and fees still owed,
        collection_status - Flag for payments in collections,
        compliance [target variable for prediction] ,
         Null = Not responsible,
         0 = Responsible, non-compliant,
         1 = Responsible, compliant,
        compliance_detail - More information on why each ticket was marked compliant or non-compliant
    
    ## Evaluation
  
    For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.
    
    Example:
    
        ticket_id,
           284932    0.531842,
           285362    0.401958,
           285361    0.105928,
           285338    0.018572,
                     ...,
           376499    0.208567,
           376500    0.818759,
           369851    0.018528,
           Name: compliance, dtype: float32

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('train.csv',encoding = 'ISO-8859-1',low_memory=False)
print(train_data.shape)
train_data.head()

(250306, 34)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [3]:
train_data = train_data[(train_data['compliance'] == 0)|(train_data['compliance'] == 1)]
train_data.shape

(159880, 34)

In [4]:
test_data = pd.read_csv('test.csv',encoding = 'ISO-8859-1',low_memory=False)
print(test_data.shape)
test_data.head()

(61001, 27)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [5]:
test_data[test_data['city'] == 'DETROIT'].shape

(26358, 27)

In [6]:
address = pd.read_csv('addresses.csv')
address.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [7]:
latlon = pd.read_csv('latlons.csv')
latlon.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [8]:
address = address.set_index('address').join(latlon.set_index('address'), how = 'left')
address.head()

,ticket_id,lat,lon
address,,,
"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
"-15126 harper, Detroit MI",344821,42.406402,-82.957525
"0 10th st, Detroit MI",24928,42.325689,-83.064330
"0 10th st, Detroit MI",71887,42.325689,-83.064330


In [9]:
train_data = train_data.set_index('ticket_id').join(address.set_index('ticket_id'))
train_data.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,42.326937,-83.135118
22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,CA,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.145257,-83.208233
18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,MI,...,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.433466,-83.023493
18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,MI,...,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.388641,-83.037858


In [10]:
test_data = test_data.set_index('ticket_id').join(address.set_index('ticket_id'))
test_data.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.426239,-83.238259
285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.309661,-83.122426
285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.308830,-83.121116


In [11]:
len(train_data[train_data['hearing_date'].isnull()])

227

In [12]:
len(train_data[train_data['ticket_issued_date'].isnull()])

0

In [13]:
train_data = train_data[~train_data['hearing_date'].isnull()]

In [14]:
len(test_data[test_data['hearing_date'].isnull()])

2197

In [15]:
len(test_data[test_data['ticket_issued_date'].isnull()])

0

In [16]:
#test_data = test_data[~test_data['hearing_date'].isnull()]

In [17]:
from datetime import datetime
def time_gap(hearing_date_str, ticket_issued_date_str):
    if not hearing_date_str or type(hearing_date_str)!=str: return 73 
    hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
    ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S" )
    gap = hearing_date - ticket_issued_date
    
    return gap.days

In [18]:
gap = datetime.strptime("2005-02-22 15:00:00", "%Y-%m-%d %H:%M:%S") - datetime.strptime("2004-06-16 12:30:00", "%Y-%m-%d %H:%M:%S")
gap.days

251

In [19]:
train_data['time_gap'] = train_data.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1) 

In [20]:
train_data['time_gap'].mean()

72.64741032113396

In [21]:
train_data.lat.fillna(method='pad', inplace=True)
train_data.lon.fillna(method='pad', inplace=True)
train_data.state.fillna(method='pad', inplace=True)

test_data.lat.fillna(method='pad', inplace=True)
test_data.lon.fillna(method='pad', inplace=True)
test_data.state.fillna(method='pad', inplace=True)

In [22]:
feature_to_be_splitted = ['agency_name', 'state', 'disposition']

train_data = pd.get_dummies(train_data, columns=feature_to_be_splitted)
test_data = pd.get_dummies(test_data, columns=feature_to_be_splitted)

In [23]:
list_to_remove_train = [
        'balance_due',
        'collection_status',
        'compliance_detail',
        'payment_amount',
        'payment_date',
        'payment_status'
    ]
list_to_remove_all = ['fine_amount', 'violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
train_data.drop(list_to_remove_train, axis=1, inplace=True)
train_data.drop(list_to_remove_all, axis=1, inplace=True)
test_data.drop(list_to_remove_all, axis=1, inplace=True)

train_data.head()

,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance,lat,lon,time_gap,...,state_VI,state_VT,state_WA,state_WI,state_WV,state_WY,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,20.0,10.0,25.0,0.0,0.0,305.0,0.0,42.390729,-83.124268,369,...,0,0,0,0,0,0,0,0,1,0
27586,20.0,10.0,75.0,0.0,0.0,855.0,1.0,42.326937,-83.135118,378,...,0,0,0,0,0,0,0,0,0,1
22046,20.0,10.0,25.0,0.0,0.0,305.0,0.0,42.145257,-83.208233,323,...,0,0,0,0,0,0,0,0,1,0
18738,20.0,10.0,75.0,0.0,0.0,855.0,0.0,42.433466,-83.023493,253,...,0,0,0,0,0,0,0,0,1,0
18735,20.0,10.0,10.0,0.0,0.0,140.0,0.0,42.388641,-83.037858,251,...,0,0,0,0,0,0,0,0,1,0


In [24]:
train_features = train_data.columns.drop('compliance')
train_features

Index(['admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'lat', 'lon', 'time_gap',
       'agency_name_Buildings, Safety Engineering & Env Department',
       'agency_name_Department of Public Works',
       'agency_name_Detroit Police Department',
       'agency_name_Health Department', 'agency_name_Neighborhood City Halls',
       'state_AK', 'state_AL', 'state_AR', 'state_AZ', 'state_BC', 'state_BL',
       'state_CA', 'state_CO', 'state_CT', 'state_DC', 'state_DE', 'state_FL',
       'state_GA', 'state_HI', 'state_IA', 'state_ID', 'state_IL', 'state_IN',
       'state_KS', 'state_KY', 'state_LA', 'state_MA', 'state_MD', 'state_ME',
       'state_MI', 'state_MN', 'state_MO', 'state_MS', 'state_MT', 'state_NB',
       'state_NC', 'state_ND', 'state_NH', 'state_NJ', 'state_NM', 'state_NV',
       'state_NY', 'state_OH', 'state_OK', 'state_ON', 'state_OR', 'state_PA',
       'state_PR', 'state_QC', 'state_QL', 'state_RI', 'state_SC', 's

In [25]:
train_features_set = set(train_features)
for feature in set(train_features):
    if feature not in test_data:
        train_features_set.remove(feature)
train_features = list(train_features_set)

In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

def BlightPrediction():
    X_train = train_data[train_features]
    y_train = train_data.compliance
    X_test = test_data[train_features]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    Model = RandomForestClassifier(n_estimators = 5,random_state=0).fit(X_train_scaled, y_train)
    
    y_test = Model.predict(X_test_scaled)
    
    test_proba = Model.predict_proba(X_test_scaled)[:,1]
    test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    test_df['compliance'] = test_proba
    test_df.set_index('ticket_id', inplace=True)

    
    print(" Accuracy on training set {:.3f}".format(Model.score(X_train_scaled, y_train)))
    print(" Accuracy on testing set {:.3f}".format(Model.score(X_test_scaled, y_test)))
    
    
    return test_df.compliance

In [27]:
BlightPrediction()

 Accuracy on training set 0.982
 Accuracy on testing set 1.000


ticket_id
284932    0.0
285362    0.0
285361    0.0
285338    0.0
285346    0.0
285345    0.2
285347    0.0
285342    0.8
285530    0.2
284989    0.0
285344    0.0
285343    0.0
285340    0.0
285341    0.0
285349    0.0
285348    0.0
284991    0.0
285532    0.0
285406    0.0
285001    0.0
285006    0.0
285405    0.0
285337    0.0
285496    0.0
285497    0.2
285378    0.0
285589    0.0
285585    0.0
285501    0.0
285581    0.0
         ... 
376367    0.0
376366    0.0
376362    0.0
376363    0.4
376365    0.0
376364    0.0
376228    0.4
376265    0.0
376286    0.4
376320    0.0
376314    0.0
376327    0.8
376385    0.2
376435    0.8
376370    1.0
376434    0.0
376459    0.0
376478    0.0
376473    0.0
376484    0.0
376482    0.0
376480    0.0
376479    0.0
376481    0.0
376483    0.0
376496    0.0
376497    0.0
376499    0.0
376500    0.0
369851    0.6
Name: compliance, Length: 61001, dtype: float64